In [1]:
import pandas as pd
from dotenv import dotenv_values
from pathlib import Path
import datasets
import pandas as pd
from scipy.special import softmax

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])

Set the iteration to one higher than the previous iteration.

i.e. set it to 0 for the beginnning, 1 for the first round etc.

In [3]:
iteration = 1

In [ ]:
data_loss = datasets.load_from_disk(base_path/f"data/processed/loss/active-learning-iteration-{iteration}")

data_unexpected = datasets.load_from_disk(base_path/f"data/processed/unexpected/active-learning-iteration-{iteration}")
annotations = pd.read_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration-1}.pkl")

df = data_loss.to_pandas().merge(data_unexpected.to_pandas()[["report_id", "paragraph_nr", f"logits_unexpected_AL_{iteration}"]], on=["report_id", "paragraph_nr"])
if "input_ids" in df.columns:
    df.drop(columns=["input_ids", "token_type_ids", "attention_mask", "text", "n_words", "filing_type"], inplace=True)
else:
    df.drop(columns=["text", "n_words", "filing_type"], inplace=True)
df[f"p_loss_AL_{iteration}"] = df[f"logits_loss_AL_{iteration}"].apply(lambda x: softmax(x)[0])
df[f"p_unexpected_AL_{iteration}"] = df[f"logits_unexpected_AL_{iteration}"].apply(lambda x: softmax(x)[0])

# Be careful with the following line as it potentially overwrites already existing labels

In [ ]:
#df.merge(annotations, on=["report_id", "paragraph_nr"]).to_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration}.pkl")